In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -e ..

Obtaining file:///home/massimiliano/Projects/fl-g13
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fl_g13 (pyproject.toml) ... done
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4783 sha256=8064e08b4f30ae57a5caf24381753035394944d203df3c3ab58bb159a922d71e
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdr_wwbp/wheels/ba/e6/76/85624c1c92362af95a88fb73ca068d24e3e74626054ee5ff32
Successfully built fl_g13
  Attempting uninstall: fl_g13
    Found existing installation: fl_g13 0.0.1
    Uninstalling fl_g13-0.0.1:
      Successfully uninstalled fl_g13-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
import dotenv
import os

dotenv.load_dotenv() # Load API Key
WANDB_API_KEY = os.getenv("WANDB_API_KEY")

wandb.login(key=WANDB_API_KEY)
from pathlib import Path

import flwr
import torch
from flwr.simulation import run_simulation
from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {device}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

from fl_g13.fl_pytorch import build_fl_dependencies
build_fl_dependencies()

# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if device == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/massimiliano/.netrc
wandb: Currently logged in as: massimilianocarli (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
2025-06-04 23:08:59.527 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu124
'vision_transformer.py' already exists.
'utils.py' already exists.


## Hyperparameters

In [5]:
from fl_g13.modeling import load_or_create
from fl_g13.editing import SparseSGDM
from torch.optim import SGD

# Model Hyper-parameters
model_config={
    "head_layers": 3,
    "head_hidden_size": 512,
    "dropout_rate": 0.0,
    "unfreeze_blocks": 0,
}

# Training Hyper-parameters
batch_size = 64
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# Model editing Hyper-parameters
model_editing = False
mask_type = 'global'
sparsity = 0.8
mask = None
model_editing_batch_size = 16

# Federated Hyper-parameters
K = 100
C = 0.1
Js = [8]
Ncs = [1, 5, 10, 50]

save_every = 5
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available

num_rounds = 200
evaluate_each = 5
partition_type = 'shard'
NUM_CLIENTS = K

# Wandb config
use_wandb = True
project_name = "FL_Dino_CIFAR100_baseline_v4"

current_path = Path.cwd()
model_save_path = current_path / f"../models/fl_dino_v4/non_iid"

## Training

In [ ]:
for Nc in Ncs:
    for J in Js:
        print('-' * 200)
        print(f"Training Non IId model")
        print(f"Nc: {Nc}, J: {J}")
        checkpoint_dir = f"{model_save_path}/{Nc}_{J}/editing"
        print(f'checkpoint_dir:{checkpoint_dir}')
        
        # Model
        model, start_epoch = load_or_create(
            path=checkpoint_dir,
            model_class=BaseDino,
            model_config=model_config,
            optimizer=None,
            scheduler=None,
            device=device,
            verbose=True,
        )
        model.to(device)

        optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
        criterion = torch.nn.CrossEntropyLoss()
        scheduler = CosineAnnealingLR(
            optimizer=optimizer,
            T_max=T_max,
            eta_min=eta_min
        )

        ## Unfreeze blocks
        num_blocks = 0
        model.unfreeze_blocks(num_blocks)
        num_shards_per_partition = Nc

        os.makedirs(checkpoint_dir, exist_ok=True)
        name = f"FL_Dino_Baseline_model_non_iid_{Nc}_{J}"
        
        wandb_config = {
            # Wandb Params
            'name': name,
            'project_name': project_name,
            'run_id': f"{name}",
            # Federated Learning param
            "fraction_fit": fraction_fit,
            'partition_type': partition_type,
            'K': K,
            'C': C,
            'J': J,
            'Nc': Nc,
            # Model editing params
            'model_editing': model_editing,
            'mask_type': mask_type,
            'sparsity': sparsity,
            'model_editing_batch_size': model_editing_batch_size,
            # Training params
            'lr': lr,
            'momentum': momentum,
        }

        if model_editing:
            # Create a dummy mask for SparseSGDM
            init_mask = [torch.ones_like(p, device=p.device) for p in
                         model.parameters()]  # Must be done AFTER the model is moved to the device
            # Optimizer, scheduler, and loss function
            optimizer = SparseSGDM(
                model.parameters(),
                mask=init_mask,
                lr=lr,
                momentum=0.9,
                weight_decay=1e-5
            )

        client = get_client_app(
            model=model,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            partition_type=partition_type,
            local_epochs=1,
            local_steps=J,
            batch_size=batch_size,
            num_shards_per_partition=num_shards_per_partition,
            scheduler=None, #! Clients wont use scheduler, as it doesnt make sense here
            verbose=0,
            model_editing=model_editing,
            mask_type=mask_type,
            sparsity=sparsity,
            mask=mask,
            model_editing_batch_size=model_editing_batch_size,
            mask_func=None
        )

        compute_round = num_rounds + 1 - start_epoch
        server = get_server_app(
            checkpoint_dir=checkpoint_dir,
            model_class=model,
            optimizer=optimizer,
            criterion=criterion,
            scheduler=scheduler,
            num_rounds=compute_round,
            fraction_fit=fraction_fit,
            fraction_evaluate=fraction_evaluate,
            min_fit_clients=min_fit_clients,
            min_evaluate_clients=min_evaluate_clients,
            min_available_clients=min_available_clients,
            device=device,
            use_wandb=use_wandb,
            wandb_config=wandb_config,
            save_every=save_every,
            prefix='fl_baseline',
            evaluate_each=evaluate_each
        )
        
        # Run simulation
        run_simulation(
            server_app=server,
            client_app=client,
            num_supernodes=NUM_CLIENTS,
            backend_config=backend_config
        )

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Training Non IId model
Nc: 1, J: 8
checkpoint_dir:/home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/1_8/editing
⚠️ No checkpoint found at /home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/1_8/editing. Creating a new model.


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


⚠️ No checkpoint found at /home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/1_8/editing. Creating a new model.
[Server] Server on device: cuda:0
[Server] CUDA available in client: True
Using strategy 'CustomFedAvg' (default option)


/home/massimiliano/miniconda3/envs/fl-g13/lib/python3.11/site-packages/wandb/analytics/sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa
/home/massimiliano/miniconda3/envs/fl-g13/lib/python3.11/site-packages/wandb/analytics/sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


INFO :      Starting Flower ServerApp, config: num_rounds=200, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


...

(ClientAppActor pid=109967) [Client] Client on device: cuda:0
(ClientAppActor pid=109967) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 2/2 [00:00<00:00,  7.73batch/s]
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 17]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)


(ClientAppActor pid=109967) [Client] Client on device: cuda:0
(ClientAppActor pid=109967) [Client] CUDA available in client: True
(ClientAppActor pid=109967) No prefix/name for the model was provided, choosen prefix/name: zesty_kakuna_51
(ClientAppActor pid=109967) 
(ClientAppActor pid=109967) Step 1/8 | Total batches: 7
(ClientAppActor pid=109967) Step 2/8 | Total batches: 7
(ClientAppActor pid=109967) Step 3/8 | Total batches: 7
(ClientAppActor pid=109967) Step 4/8 | Total batches: 7
(ClientAppActor pid=109967) Step 5/8 | Total batches: 7
(ClientAppActor pid=109967) Step 6/8 | Total batches: 7
(ClientAppActor pid=109967) Step 7/8 | Total batches: 7
(ClientAppActor pid=109967) Step 8/8 | Total batches: 7
(ClientAppActor pid=109967) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=109967) 	📊 Training Loss: 0.7645
(ClientAppActor pid=109967) 	✅ Training Accuracy: 82.11%
(ClientAppActor pid=109967) 	⏳ Elapsed Time: 1.18s | ETA: 0.00s
(ClientAppActor pid=109967) 	🕒 Completed At: 23:17


INFO :      aggregate_fit: received 10 results and 0 failures


(ClientAppActor pid=109967) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=109967) 	📊 Training Loss: 1.0057
(ClientAppActor pid=109967) 	✅ Training Accuracy: 77.37%
(ClientAppActor pid=109967) 	⏳ Elapsed Time: 1.11s | ETA: 0.00s
(ClientAppActor pid=109967) 	🕒 Completed At: 23:17
(ClientAppActor pid=109967) 
